In [1]:
from torch.nn.utils.rnn import pad_sequence
import math
import torchtext
import torch
import torch.nn as nn
from torch import Tensor
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
from torchtext.vocab import Vocab
from torch.nn import TransformerEncoder, TransformerDecoder, TransformerEncoderLayer, TransformerDecoderLayer
import io
import time
import numpy as np
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
from nltk.translate.bleu_score import sentence_bleu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"([.!?])", r" \1", s)
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [3]:
def readText(path, reverse=False):
    # Read the file and split into lines
    orgfile = open(path, encoding='utf-8').\
        read().strip().split('\n')

    lines = []
    for line in orgfile:
      lines.append(line.split("CC-BY", 1)[0].strip())

    # Split every line into pairs and normalize
    pairs = [[s for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
    return pairs

In [4]:
path = 'C:/Users/sanzh/Desktop/english-japanese.txt'
pairs = readText(path, True)
japanese = [item[0] for item in pairs]
english = [item[1] for item in pairs]

In [5]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List

In [6]:
SRC_LANGUAGE = 'ja'
TGT_LANGUAGE = 'en'
token_transform = {}
vocab_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='ja_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

In [7]:
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
ja_tokenizer = get_tokenizer('spacy', language='ja_core_news_sm')

In [8]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [9]:
def build_vocab(data, tokenizer):
    counter = Counter()
    for sentence in data:
        counter.update(tokenizer(sentence))
    return torchtext.vocab.vocab(counter, specials = special_symbols)

In [10]:
en_vocab = build_vocab(english, en_tokenizer)
ja_vocab = build_vocab(japanese, ja_tokenizer)

In [11]:
def data_process(src, tg, src_tokenizer, tg_tokenizer, src_vocab, tg_vocab):
  data = []
  for (raw_src, raw_tg) in zip(src, tg):
    src_tensor_ = torch.tensor([src_vocab[token] for token in src_tokenizer(raw_src.rstrip("\n"))],
                            dtype=torch.long)
    tg_tensor_ = torch.tensor([tg_vocab[token] for token in tg_tokenizer(raw_tg.rstrip("\n"))],
                            dtype=torch.long)
    data.append((src_tensor_, tg_tensor_))
  return data

In [12]:
train_data = data_process(japanese, english, ja_tokenizer, en_tokenizer, ja_vocab, en_vocab)

In [13]:
BATCH_SIZE = 16
PAD_IDX = ja_vocab['<pad>']
BOS_IDX = ja_vocab['<bos>']
EOS_IDX = ja_vocab['<eos>']

def generate_batch(data_batch):
  src_batch, tg_batch = [], []
  for (src_item, tg_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
    tg_batch.append(torch.cat([torch.tensor([BOS_IDX]), tg_item, torch.tensor([EOS_IDX])], dim=0))
  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  tg_batch = pad_sequence(tg_batch, padding_value=PAD_IDX)
  return src_batch, tg_batch

In [14]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)

In [15]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self, num_encoder_layers: int, num_decoder_layers: int,
                 emb_size: int, src_vocab_size: int, tgt_vocab_size: int,
                 dim_feedforward:int = 512, dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        decoder_layer = TransformerDecoderLayer(d_model=emb_size, nhead=NHEAD,
                                                dim_feedforward=dim_feedforward)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout=dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None,
                                        tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self, emb_size: int, dropout, maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [17]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

  src_padding_mask = (src == PAD_IDX).transpose(0, 1)
  tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
  return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [14]:
SRC_VOCAB_SIZE = len(ja_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 16
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 16
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(device)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

optimizer = torch.optim.Adam(
    transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9
)

In [60]:
def train_epoch(model, train_iter, optimizer):
  model.train()
  losses = 0
  for idx, (src, tgt) in enumerate(train_iter):
      src = src.to(device)
      tgt = tgt.to(device)

      tgt_input = tgt[:-1, :]

      src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

      logits = model(src, tgt_input, src_mask, tgt_mask,
                                src_padding_mask, tgt_padding_mask, src_padding_mask)

      optimizer.zero_grad()

      tgt_out = tgt[1:,:]
      loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
      loss.backward()

      optimizer.step()
      losses += loss.item()
  return losses / len(train_iter)

In [18]:
def evaluate(model, val_iter):
  model.eval()
  losses = 0
  for idx, (src, tgt) in (enumerate(valid_iter)):
    src = src.to(device)
    tgt = tgt.to(device)

    tgt_input = tgt[:-1, :]

    src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask,
                              src_padding_mask, tgt_padding_mask, src_padding_mask)
    tgt_out = tgt[1:,:]
    loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    losses += loss.item()
  return losses / len(val_iter)

In [19]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        memory_mask = torch.zeros(ys.shape[0], memory.shape[0]).to(device).type(torch.bool)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
          break
    return ys

In [20]:
def translate(model, src, src_vocab, tgt_vocab, src_tokenizer):
  model.eval()
  tokens = [BOS_IDX] + [src_vocab.get_stoi()[tok] for tok in src_tokenizer(src)]+ [EOS_IDX]
  num_tokens = len(tokens)
  src = (torch.LongTensor(tokens).reshape(num_tokens, 1) )
  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
  return " ".join([tgt_vocab.get_itos()[tok] for tok in tgt_tokens]).replace("<bos>", "").replace("<eos>", "")

In [64]:
for epoch in range(1, NUM_EPOCHS+1):
  start_time = time.time()
  train_loss = train_epoch(transformer, train_iter, optimizer)
  end_time = time.time()
  print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, "
          f"Epoch time = {(end_time - start_time):.3f}s"))

Epoch: 1, Train loss: 3.793, Epoch time = 143.390s
Epoch: 2, Train loss: 2.783, Epoch time = 141.049s
Epoch: 3, Train loss: 2.297, Epoch time = 141.877s
Epoch: 4, Train loss: 2.004, Epoch time = 142.344s
Epoch: 5, Train loss: 1.803, Epoch time = 142.749s
Epoch: 6, Train loss: 1.653, Epoch time = 142.889s
Epoch: 7, Train loss: 1.535, Epoch time = 139.989s
Epoch: 8, Train loss: 1.433, Epoch time = 139.947s
Epoch: 9, Train loss: 1.346, Epoch time = 139.916s
Epoch: 10, Train loss: 1.280, Epoch time = 140.856s
Epoch: 11, Train loss: 1.256, Epoch time = 140.632s
Epoch: 12, Train loss: 1.220, Epoch time = 140.393s
Epoch: 13, Train loss: 1.180, Epoch time = 140.550s
Epoch: 14, Train loss: 1.148, Epoch time = 140.824s
Epoch: 15, Train loss: 1.117, Epoch time = 140.662s
Epoch: 16, Train loss: 1.091, Epoch time = 140.044s


In [66]:
path = "C:\\Users\\sanzh/Desktop/translator.pt"
torch.save(transformer.state_dict(), path)

In [21]:
SRC_VOCAB_SIZE = len(ja_vocab)
TGT_VOCAB_SIZE = len(en_vocab)
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 16
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 16
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS,
                                 EMB_SIZE, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE,
                                 FFN_HID_DIM)
transformer = transformer.to(device)

In [22]:
path = "C:\\Users\\sanzh/Desktop/translator.pt"
transformer.load_state_dict(torch.load(path))

<All keys matched successfully>

In [23]:
translate(transformer, "嘘は続くでしょう。", ja_vocab, en_vocab, ja_tokenizer)

' You will have a lie . '

In [12]:
japanese_10 = [phrase for phrase in japanese if len(phrase) < 11]
japanese_10_indices = [japanese.index(phrase) for phrase in japanese_10]
english_10 = [english[i] for i in japanese_10_indices]

In [13]:
len(japanese_10)

14341

In [27]:
def evaluate_random_bleu_score():
    i = random.choice(range(len(japanese_10)))
    japanese_phrase = japanese_10[i]
    english_phrase = english_10[i]
    translated = translate(transformer, japanese_phrase, ja_vocab, en_vocab, ja_tokenizer)
    reference = en_tokenizer(english_phrase)
    hypothesis = en_tokenizer(translated)
    print("Input Phrase: " + japanese_phrase)
    print()
    print("Transformer Translation: " + translated)
    print()
    print("Actual Translation: " + english_phrase)
    print()
    print("Bleu Score: ")
    return sentence_bleu([reference], hypothesis, weights = [1])

In [29]:
evaluate_random_bleu_score()

Input Phrase: 私はネコ派です。

Transformer Translation:  I am a cat person . 

Actual Translation: I am a cat person.

Bleu Score: 


0.8571428571428571

In [87]:
res = 0
for i in range(100):
    res += evaluate_random_bleu_score()
print(res/100)

Input Phrase: すごいですね。

Transformer Translation:  It 's very nice . 

Actual Translation: It's amazing.

Bleu Score: 
Input Phrase: そこに誰かいますか？

Transformer Translation:  Is anyone in there ? 

Actual Translation: Is there anyone there?

Bleu Score: 
Input Phrase: 明日って祝日だっけ？

Transformer Translation:  Is tomorrow a holiday ? 

Actual Translation: Is tomorrow a holiday?

Bleu Score: 
Input Phrase: 部屋を片づけて。

Transformer Translation:  Put the room away . 

Actual Translation: Clean the room.

Bleu Score: 
Input Phrase: 誰かに聞いてみて。

Transformer Translation:  Ask anybody . 

Actual Translation: Ask anyone.

Bleu Score: 
Input Phrase: 離婚したいんだ。

Transformer Translation:  I want to get divorced . 

Actual Translation: I want to get divorced.

Bleu Score: 
Input Phrase: 冷蔵庫が開いてるよ。

Transformer Translation:  The refrigerator is open . 

Actual Translation: The refrigerator is open.

Bleu Score: 
Input Phrase: 私は急いで服を着た。

Transformer Translation:  I got dressed in a hurry . 

Actual Translation: I go

Input Phrase: これは直せる。

Transformer Translation:  This can be possible . 

Actual Translation: This can be fixed.

Bleu Score: 
Input Phrase: さあ楽しんで。

Transformer Translation:  Now have fun . 

Actual Translation: Now have fun.

Bleu Score: 
Input Phrase: 肌が乾燥しています。

Transformer Translation:  I have my skin about . 

Actual Translation: I have dry skin.

Bleu Score: 
Input Phrase: 他に何か見ましたか？

Transformer Translation:  Did you see anything else ? 

Actual Translation: Did you see anything else?

Bleu Score: 
Input Phrase: 無料なのですか。

Transformer Translation:  Is it free ? 

Actual Translation: Is it free?

Bleu Score: 
Input Phrase: 私は学生です。

Transformer Translation:  I 'm a student . 

Actual Translation: I'm a student.

Bleu Score: 
Input Phrase: 毎朝、体調が悪いの。

Transformer Translation:  I do n't feel well every morning . 

Actual Translation: I feel sick every morning.

Bleu Score: 
Input Phrase: トムは馬に噛まれた。

Transformer Translation:  Tom got bitten by a horse . 

Actual Translation: Tom got 

In [89]:
res = 0
for i in range(100):
    res += evaluate_random_bleu_score()
print(res/100)

0.6235958435875854


In [90]:
res = 0
for i in range(100):
    res += evaluate_random_bleu_score()
print(res/100)

0.6056413124699026
